In [3]:
# !pip install datarobot

## Taking an LLM "into" DataRobot

### prequisites

This example relies on Gpt4o, so the requirement is that you have access to Gpt4o.  This example shows 

* the use of Azure OpenAI endpoint that is serving Gpt4o for the purposes of inference.  

In order to utilize required credentials,[DataRobot credential manager](https://docs.datarobot.com/en/docs/data/connect-data/stored-creds.html#credentials-management) is being used to keep the Api token.  


In [ ]:
import pandas as pd 
from custom import *
data = pd.DataFrame([
    dict(promptText = "tell me a joke"),
    dict(promptText = "what's up doc?  can we rock"),
    dict(promptText = "describe bolzano weierstrauss theorem"),
    ])
data.to_csv("test_data.csv")

In [ ]:
!pip install datarobot-drum 
!export TARGET_NAME=resultText && drum score --code-dir ./model --target-type textgeneration --runtime-params runtime_params.yaml --input test_data.csv

,resultText
0,"Sure, here's a joke for you:\n\nWhy don't skel..."
1,It sounds like you're referencing the classic ...
2,The Bolzano-Weierstrass theorem is a fundament...


## Take this into datarobot

Pick the appropriate datarobot custom model environment

In [1]:
import datarobot as dr
environment = [env for env in dr.ExecutionEnvironment.list() if env.name == "[DataRobot] Python 3.11 GenAI"].pop()
environment


ExecutionEnvironment('[DataRobot] Python 3.11 GenAI')

Create the list of required runtime parameters (see `model-metadata.yaml` for details)
What follows is a list of credentials already being managed by DataRobot.  

In [2]:
credentials = [c for c in dr.Credential.list() if c.name in  ["DR_OPENAI_API_KEY", "Langchain API Key"]]
credentials

[Credential('662811a051f136d67f7997bd', 'DR_OPENAI_API_KEY', 'api_token'),
 Credential('676332535bccdbe998577038', 'Langchain API Key', 'api_token')]

We'll grab the credential ids from the credential list above and set up the run time parameters

In [3]:
runtime_parameter_values = [
    dr.models.runtime_parameters.RuntimeParameterValue(field_name = "AZURE_OPENAI_API_KEY", type = "credential", value = "662811a051f136d67f7997bd"),
    dr.models.runtime_parameters.RuntimeParameterValue(field_name = "LANGCHAIN_API_KEY", type = "credential", value = "676332535bccdbe998577038"),
    ]

Create the custom model in the custom model workshop

In [4]:
cim = dr.CustomInferenceModel.create(name = "Langchain Traced Gpt4o", 
                                    target_type = dr.enums.TARGET_TYPE.TEXT_GENERATION, 
                                    target_name = "resultText", 
                                    network_egress_policy=dr.enums.NETWORK_EGRESS_POLICY.PUBLIC)


add a version with all necessary assets

In [6]:
cimv = dr.CustomModelVersion.create_clean(custom_model_id = cim.id, 
                                          base_environment_id = environment.id, 
                                          runtime_parameter_values = runtime_parameter_values,                                        
                                          files = [("custom.py", "custom.py"),
                                                   ("model-metadata.yaml", "model-metadata.yaml"),
                                                   ("requirements.txt", "requirements.txt")])
## or 
# cim = dr.CustomInferenceModel.get("67350386bab555bb07182a07")
# cimv = dr.CustomModelVersion.create_from_previous(custom_model_id = cim.id, 
#                                                    base_environment_id = environment.id,
#                                                   files = [("custom.py", "custom.py"),
#                                                    ("model-metadata.yaml", "model-metadata.yaml"),
#                                                    ("requirements.txt", "requirements.txt")])
                                        

build the custom model environment (since there was a `requirements.txt` present).  This could take some time

In [7]:
build = dr.CustomModelVersionDependencyBuild.start_build(cim.id, cimv.id, max_wait = 1200)

Register your test dataset

In [9]:
llm_dataset_test = dr.Dataset.create_from_in_memory_data(data_frame = data)
## or 
# llm_dataset_test = dr.Dataset.get("<dataset-id>")

Test the LLM proxy in DataRobot.  This could take some time.  

You should be able to pull up langsmith and check tracing depending on how you set the runtime parameters

In [10]:
custom_model_test = dr.CustomModelTest.create(cim.id, cimv.id, dataset_id = llm_dataset_test.id, network_egress_policy = dr.enums.NETWORK_EGRESS_POLICY.PUBLIC)

In [11]:
custom_model_test.overall_status

'succeeded'

## register model

In [12]:

## register the custom model version in the dr model registry 
registered_model_version = dr.RegisteredModelVersion.create_for_custom_model_version(
    custom_model_version_id =  cimv.id, 
    name = "Traced GPT4o", 
    description = "GPT4o traced via langsmith.  Additional arguments in the payload are required to enable tracing", 
    registered_model_name = "Traced GPT4o", 
    registered_model_description = "GPT4o traced via langsmith.  Additional arguments in the payload are required to enable tracing"
)

Create a deployment that will be used in the playground

In [13]:
dr.PredictionEnvironment.list()

[PredictionEnvironment('612f829f8cfc3e5e9ca6b35d', 'Algorithmia', 'other', ''),
 PredictionEnvironment('613bb1cba2c312aa4c43d0ca', 'algosales.productionize.ai', 'other', 'algorithmia'),
 PredictionEnvironment('60904e2b109ff4d7bfe0be10', 'cola', 'other', 'None'),
 PredictionEnvironment('66a929580c3e174abc7542cb', 'DataRobot Serverless', 'datarobotServerless', ''),
 PredictionEnvironment('668401df486fcb136bf056d1', 'DataRobot Serverless Predictions', 'datarobotServerless', 'DataRobot Serverless Predictions'),
 PredictionEnvironment('61f160a0bfd3a9c426f7e60c', 'Default', 'aws', 'None'),
 PredictionEnvironment('6786861249482a46f808be18', 'Dev PLan', 'datarobotServerless', ''),
 PredictionEnvironment('65fb005b00e2ba108b8758d0', 'External Prediction Environment', 'other', 'Prediction environment for external LLM apps using DR Guardrails - created via drx'),
 PredictionEnvironment('65faf040496d866fe2a78081', 'External Prediction Environment', 'other', 'Prediction environment for external LLM 

In [15]:
prediction_server = [ pe for pe in dr.PredictionEnvironment.list() if pe.name == "DataRobot Serverless"].pop()

In [17]:
client = dr.Client() 
payload = {
    "modelPackageId":registered_model_version.id,
    "predictionEnvironmentId": prediction_server.id,
    "label":"Traced GPT4o",
    "importance":"LOW"}
dep_req = client.post("deployments/fromModelPackage", data = payload)

In [20]:

## wait for the deployment to create
response = client.get(dep_req.headers["location"].replace("https://app.datarobot.com/api/v2/","")).json()
while response.get("status") == "INITIALIZED":
    response = client.get(dep_req.headers["location"].replace("https://app.datarobot.com/api/v2/","")).json()
response = client.get(dep_req.headers["location"].replace("https://app.datarobot.com/api/v2/","")).json()
deployment_id = response["id"]
print(response)

{'id': '678694c9e33806cb8599a60f', 'label': 'Traced GPT4o', 'description': None, 'createdAt': '2025-01-14T16:46:07.434000Z', 'model': {'id': '6786929e32fb5850bccfa601', 'type': 'Langchain Traced Gpt4o', 'targetName': 'resultText', 'targetType': 'TextGeneration', 'projectId': None, 'projectName': None, 'unsupervisedMode': False, 'unstructuredModelKind': False, 'buildEnvironmentType': 'Python', 'deployedAt': '2025-01-14T16:46:07.503000Z', 'customModelImage': {'customModelId': '6786929a32fb5850bccfa5fe', 'customModelName': 'Langchain Traced Gpt4o', 'customModelVersionId': '6786929e32fb5850bccfa601', 'customModelVersionLabel': 'v1.0', 'executionEnvironmentId': '64d2ba178dd3f0b1fa2162f0', 'executionEnvironmentName': '[DataRobot] Python 3.11 GenAI', 'executionEnvironmentVersionId': '6717f8e0f7c696f62730c286', 'executionEnvironmentVersionLabel': 'v12'}, 'isDeprecated': False, 'prompt': None}, 'status': 'active', 'capabilities': {'supportsModelReplacement': True, 'supportsTargetDriftTracking':

In [22]:
deployment = dr.Deployment.get(deployment_id)

## Check that it is working as expected

In [29]:
prediction_server.platform

'datarobotServerless'

In [35]:
import requests
import os
import json
import datarobot as dr 
import pprint
# deployment = dr.Deployment.get("673505fb8a477d1f2fbaed3b")
# URL = f'{deployment.prediction_environment["name"]}/predApi/v1.0/deployments/{deployment.id}/predictions'
if prediction_server.platform == "datarobotServerless":
    URL = f"https://app.datarobot.com/api/v2/deployments/{deployment.id}/predictions"
    datarobot_key = "placeholder"
else:
    URL =  f'{deployment.prediction_environment["name"]}/predApi/v1.0/deployments/{deployment.id}/predictions'
    datarobot_key = deployment.default_prediction_server.get("datarobot-key")
headers = {
    'Content-Type': 'text/plain; charset=UTF-8',
    'Authorization': 'Bearer {}'.format(os.environ["DATAROBOT_API_TOKEN"]),
    'DataRobot-Key': datarobot_key
}
query = "what's up doc?"
response = requests.post( URL, headers = headers, data = data.to_csv(index = False))
pprint.pprint(response.json())


{'data': [{'deploymentApprovalStatus': 'APPROVED',
           'prediction': "Sure, here's one for you:\n"
                         '\n'
                         "Why don't skeletons fight each other?\n"
                         '\n'
                         "They don't have the guts!",
           'predictionValues': [{'label': 'resultText',
                                 'value': "Sure, here's one for you:\n"
                                          '\n'
                                          "Why don't skeletons fight each "
                                          'other?\n'
                                          '\n'
                                          "They don't have the guts!"}],
           'rowId': 0},
          {'deploymentApprovalStatus': 'APPROVED',
           'prediction': 'Sure thing! "What\'s up, Doc?" is a classic '
                         'catchphrase associated with Bugs Bunny, and "Can We '
                         'Rock?" sounds like you\'re ready to 

In [36]:
data

,promptText,trace_prompt,LANGCHAIN_PROJECT
0,tell me a joke,False,NaN
1,what's up doc? can we rock,NaN,NaN
2,describe bolzano weierstrauss theorem,NaN,math help v4


In [34]:
if prediction_server.platform == "datarobotServerless":
    URL = f"https://app.datarobot.com/api/v2/deployments/{deployment.id}/predictions"
    datarobot_key = "placeholder"
else:
    URL =  f'{deployment.prediction_environment["name"]}/predApi/v1.0/deployments/{deployment.id}/predictions'
    datarobot_key = deployment.default_prediction_server.get("datarobot-key")
headers = {
    'Content-Type': 'application/json; charset=UTF-8',
    'Authorization': 'Bearer {}'.format(os.environ["DATAROBOT_API_TOKEN"]),
    'DataRobot-Key': datarobot_key
}
query = "what's up doc?"
response = requests.post( URL, headers = headers, 
                         data = json.dumps( [
                             {"promptText": "tell me a joke", "trace_prompt": True, "LANGCHAIN_PROJECT": "comedian"}
                             ]))
pprint.pprint(response.json())


{'data': [{'deploymentApprovalStatus': 'APPROVED',
           'prediction': "Sure, here's a joke for you:\n"
                         '\n'
                         "Why don't skeletons fight each other?\n"
                         '\n'
                         "They don't have the guts!",
           'predictionValues': [{'label': 'resultText',
                                 'value': "Sure, here's a joke for you:\n"
                                          '\n'
                                          "Why don't skeletons fight each "
                                          'other?\n'
                                          '\n'
                                          "They don't have the guts!"}],
           'rowId': 0}]}


## Lasty, 

This last piece will make our LLM available in playground.  Given the way we instrumented tracing in out custom.py file, all prompting done in playground with this llm will be traced and available in the default project for langsmith. 

In [30]:
custom_model_llm_validation = dr.genai.CustomModelLLMValidation.create(
        prompt_column_name="promptText",
        target_column_name="resultText",
        deployment_id=deployment.id,
        wait_for_completion=True, 
        name = deployment.__str__()
    )

assert custom_model_llm_validation.validation_status == "PASSED"